In [ ]:
library(dplyr)
library(data.table)

In [2]:
data = fread("compressed_data.csv.gz") %>% 
    filter(!is.na(prcod)) %>% 
        mutate(datadate = as.Date(datadate, "%d/%m/%Y"))
head(data)



tic,datadate,conm,exchg,sic,cshtrd,prccd,prchd,prcld,prcod,gvkey
<chr>,<date>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
PNW,2023-01-03,PINNACLE WEST CAPITAL CORP,11,4911,1442534,74.63,76.4125,73.380,76.25,1075
PNW,2023-01-04,PINNACLE WEST CAPITAL CORP,11,4911,954218,75.39,76.0950,74.630,75.10,1075
PNW,2023-01-05,PINNACLE WEST CAPITAL CORP,11,4911,994775,73.65,75.0950,73.305,74.88,1075
PNW,2023-01-06,PINNACLE WEST CAPITAL CORP,11,4911,729808,75.46,76.0200,74.480,74.49,1075
PNW,2023-01-09,PINNACLE WEST CAPITAL CORP,11,4911,656127,75.55,76.4800,75.240,75.24,1075
PNW,2023-01-10,PINNACLE WEST CAPITAL CORP,11,4911,763254,75.65,75.6950,74.880,75.31,1075


In [3]:
# 1. How many unique tickers are in your data?
unique_tic_n = length(unique(data$tic))
paste("There are", unique_tic_n, "unique tickers.")

# 2. How many unique companies are in your data?
unique_conm_n = length(unique(data$conm))
paste("There are", unique_conm_n, "unique company names.")

# 3. Display the top 5 companies by largest mean trading volume, in a table.
# calc mean trading vol by tic
data_unique = data %>%
    group_by(tic) %>%
    summarise(mean_trading_v = mean(cshtrd, na.rm = TRUE)) %>%
    ungroup() %>%
        arrange(desc(mean_trading_v))
data_unique[1:5,]

[1] "There are 502 unique tickers."

[1] "There are 499 unique company names."

tic,mean_trading_v
<chr>,<dbl>
TSLA,115314383
NVDA,113131835
PLTR,60056251
AAPL,57736403
AMD,57143415
